In [24]:
# Designed by ARH.
ticker = 'BBVA';

import os
from urllib.request import urlopen, Request
# We define the global url from investing equities
URL_INIT = 'https://es.investing.com/equities/'

# We define a dictionary that contains the tickers as well as their corresponding dividend sites.
ticker_dict = {
    'IBE': 'iberdrola-dividends/',
    'BBVA': 'bbva-dividends/',
    'ELE': 'endesa-dividends/'
}

# We define the url and the agent that will allow to download the html file.
url = os.path.join(URL_INIT, ticker_dict[ticker])
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
reg_url = url

# We make the request petition to investing.
req = Request(url=reg_url, headers=headers) 
string = urlopen(req).read()




In [25]:
# !pip install bs4
# We use BeautifulSoup to parse the html information delivered by the website.
from bs4 import BeautifulSoup
soup = BeautifulSoup(string, 'html.parser')
string_decoded = string.decode('utf-8');

In [26]:
'''
We locallize the table name directly from the html text data. The identifier will be
'<table id="dividendsHistoryDat', which is the name given to the table that contains 
dividend data.
'''
# We essentially look for our string line by line in the html text.
for i in string_decoded.split('\n'):
    if '<table id="dividendsHistoryDat' in i:
        line = i;
        
# Once we find its real identifier, we use it to get the table with all its content.
table_id = line.split('"')[1];
table = soup.find(id=table_id);

In [27]:
# We get the different cells available at the table
data_list = table.findAll('tr');

'''
We read the table content and store it in a dictionary. We store both the quantity and
the pay day. 
'''
pay_info = [];
for i in data_list[1:]:
    pay_info.append({
        'payday': i.findAll('td')[3].text,            
        'quantity': float(i.findAll('td')[1].text.replace(',','.')),
               })
pay_info


[{'payday': '15.10.2019', 'quantity': 0.26},
 {'payday': '10.04.2019', 'quantity': 0.26},
 {'payday': '13.06.2018', 'quantity': 0.3839},
 {'payday': '10.04.2018', 'quantity': 0.24},
 {'payday': '10.10.2017', 'quantity': 0.3786},
 {'payday': '26.04.2017', 'quantity': 0.3663}]

In [28]:
import csv
csv_file = "{0}_dividends.csv".format(ticker)
csv_columns = ['quantity', 'payday']
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in pay_info:
            writer.writerow(data)
except IOError:
    print("I/O error") 